### Multidomain Mesh Generation with fTetWild
 
In this notebook, we demonstrate how to generate a multidomain idealized brain mesh using boolean operations on surface meshes (STLs) with [fTetWild](https://wildmeshing.github.io/ftetwild/).

First, we import the relevant packages and set the ids of the subdomains and boundaries we want to mark:

In [ ]:
import wildmeshing as wm
import pyvista as pv
import numpy as np
import meshio
import json
from pathlib import Path

# domain ids
porous_id = 1
fluid_id = 2

# facet ids
skull_id = 1
spinal_canal_id = 2
interface_id = 3
spinal_cord_id = 4
aqueduct_V4_id = 5

As our geometry consists of multiple spheres and cylinders, we create surface meshes for each of the components with PyVista and plot them to check the setup: 

In [ ]:
skull = pv.Sphere(radius = 0.08)
parenchyma = pv.Sphere(radius = 0.07)
ventricle = pv.Sphere(radius = 0.02)
canal = pv.Cylinder(center=(0, 0, -0.105), direction=(0,0,-1),
                        radius=0.025, height=0.08).triangulate()
cord = pv.Cylinder(center=(0, 0, -0.105), direction=(0,0,-1),
                        radius=0.017, height=0.08).triangulate()
aqueduct = pv.Cylinder(center=(0, 0.03, -0.03), direction=(0,1,-1),
                       radius=0.004, height=0.06).triangulate()

# display the geometry using PyVista
pl = pv.Plotter()
pl.add_mesh(parenchyma, opacity=.6, color="blue")
pl.add_mesh(ventricle,  color="red")
pl.add_mesh(aqueduct, opacity=0.7, color="red")
pl.add_mesh(skull, opacity=0.2)
pl.add_mesh(canal, opacity=0.2)
pl.add_mesh(cord, opacity=0.7, color="blue")
pl.background_color = "white"
pl.show()

In [ ]:
# save the surface meshes for later use by fTetWild
stl_directory = Path("mesh/stls/")
stl_directory.mkdir(exist_ok=True, parents=True)
parenchyma.save(stl_directory / "parenchyma.stl")
skull.save(stl_directory / "skull.stl")
cord.save(stl_directory / "cord.stl")
canal.save(stl_directory / "canal.stl")
ventricle.save(stl_directory / "ventricle.stl")
aqueduct.save(stl_directory / "aqueduct.stl")

Now, we have everything in place to define a constructive solid geometry(CSG)-tree, describing the order and type of boolean operations carried out on the surface meshes:

* first, we take the union of brain tissue components: parenchyma and spinal cord 
* then, we subtract the ventricle and aqueduct parts from the tissue
* finally, we take the union of the outermost sphere and cylinder with the result of the previous steps

Further, we specify maximum distance between the input surfaces and the surface of the generated volumetric mesh with the *epsilon* parameter (specifying the relative envelop size) and relative edge length with *edge_length_r*.

Note that while we use a simple geometry here, we can easily replace the STL files with complex shapes and apply the same procedure to generate image-derived realistic models.

In [ ]:
tetra = wm.Tetrahedralizer(epsilon=0.002, edge_length_r=0.05,
                           coarsen=False)
tetra.load_csg_tree(json.dumps(
    {"operation":"union",
     "right":
          {
          "operation":"difference",
          "left":{"operation":"union", "left": str(stl_directory / "parenchyma.stl"),
                              "right": str(stl_directory / "cord.stl")},
          "right":{"operation":"union", "left": str(stl_directory / "aqueduct.stl"),
                          "right":str(stl_directory / "ventricle.stl")},
          },
     "left":
        {"operation":"union", "left": str(stl_directory / "skull.stl"),
                          "right":str(stl_directory / "canal.stl")},                
    }
))
tetra.tetrahedralize()
point_array, cell_array, marker = tetra.get_tet_mesh()

Next we check the *marker* values returned by *fTetWild* and set our own subdomain ids and write the mesh to file:

In [ ]:
subdomains = np.copy(marker)
subdomains[np.isin(marker, [1,2])] = fluid_id
subdomains[np.isin(marker, [3,4])] = porous_id
subdomains[np.isin(marker, [5])] = 3
subdomains[np.isin(marker, [6])] = 4
labels = np.copy(subdomains)
subdomains[np.isin(subdomains, [3,4])] = fluid_id
meshio_mesh = meshio.Mesh(point_array, [("tetra", cell_array)],
                       cell_data={"gmsh:physical": [marker.ravel()],
                                  "labels": [labels.ravel()],
                                  "subdomains": [subdomains.ravel()]})
meshio_mesh.write("mesh/mesh.xdmf")

Finally, we check the resulting mesh by reading it back in and plotting it:

In [ ]:
pv_mesh = pv.read("mesh/mesh.xdmf")
pv_mesh.clip(normal=(1,0,0),crinkle=True).plot("subdomains" ,show_edges=True, background="white")

### Generating subdomain restrictions for Multiphenics

To create function spaces restricted to a subdomain of the whole mesh with *MultiPhenics*, we need to create and save so-called *MeshRestrictions* prior to the main simulation run:

In [ ]:
from fenics import *
from multiphenics import *

def generate_subdomain_restriction(mesh, subdomains, subdomain_id):
    D = mesh.topology().dim()
    # Initialize empty restriction
    restriction = MeshRestriction(mesh, None)
    for d in range(D + 1):
        mesh_function_d = MeshFunction("bool", mesh, d)
        mesh_function_d.set_all(False)
        restriction.append(mesh_function_d)
    # Mark restriction mesh functions based on subdomain id
    for c in cells(mesh):
        if subdomains[c] == subdomain_id:
            restriction[D][c] = True
            for d in range(D):
                for e in entities(c, d):
                    restriction[d][e] = True
    # Return
    return restriction

In [ ]:
fenics_mesh = Mesh()
with XDMFFile("mesh/mesh.xdmf") as f:
    f.read(fenics_mesh)
    sm = MeshFunction("size_t", fenics_mesh, 3, 0)
    labels = MeshFunction("size_t", fenics_mesh, 3, 0)
    f.read(sm, "subdomains")
    f.read(labels, "labels")
    sm.rename("subdomains","")

fluid_restr = generate_subdomain_restriction(fenics_mesh, sm, fluid_id)
porous_restr = generate_subdomain_restriction(fenics_mesh, sm, porous_id)
fluid_restr._write("mesh/fluid.rtc.xdmf")
porous_restr._write("mesh/porous.rtc.xdmf")

## Marking Boundaries and Interfaces
We mark both the internal interface by looping over all facets in the mesh, checking the subdomain ids of the corresponding cells and identifying the internal interfaces.
In addition to the tissue-CSF interface, we also mark the surface between the aqueduct and the fourth ventricle, since we would like to compute aqueduct flow rates in the post-processing step.

In [ ]:
bm = MeshFunction("size_t", fenics_mesh, 2, 0)

def mark_internal_interface(mesh, subdomains, bm, interface_id,
                            doms=None):
    # set internal interface
    for f in facets(mesh):
        domains = []
        for c in cells(f):
            domains.append(subdomains[c])
        domains = set(domains)
        if len(domains)==2:
            if doms is None:
                bm[f] = interface_id
            elif set(doms)==domains:
                bm[f] = interface_id

# mark tissue-CSF interface
mark_internal_interface(fenics_mesh, sm, bm, interface_id)
# mark aqueduct-fourth ventricle interface for later flow computation
mark_internal_interface(fenics_mesh, labels, bm, aqueduct_V4_id, doms=[3,4])

Similarly, we mark the external boundaries by a combination of the facet location and the subdomain id of the corresponding subdomain:

In [ ]:
def mark_external_boundaries(mesh, subdomains, boundaries, 
                             subdomain_ids, new_boundary_id):
    for f in facets(mesh):
        if not f.exterior():
            continue
        c = list(cells(f))[0]
        if subdomains[c] in subdomain_ids:
            boundaries[f] = new_boundary_id
                
z_min = fenics_mesh.coordinates()[:,2].min()
bottom = CompiledSubDomain("on_boundary && near(x[2], v, tol)",
                        v=z_min, tol=0.001)
outer = CompiledSubDomain("on_boundary")
outer.mark(bm, skull_id)
bottom.mark(bm, spinal_canal_id)
mark_external_boundaries(fenics_mesh, sm, bm, 
                         [porous_id], spinal_cord_id)

with XDMFFile("mesh/facets.xdmf") as f:
    f.write(bm)
with XDMFFile("mesh/mesh.xdmf") as f:
    f.write(sm)